In [35]:
import boto3
import s3fs
import rasterio
import xarray as xr
import rioxarray
from rasterio.plot import show
from pystac_client import Client

from requests.adapters import HTTPAdapter
from urllib3 import Retry
from pystac_client.stac_api_io import StacApiIO

In [2]:
client_s3 = boto3.client(service_name='s3')

fs = s3fs.S3FileSystem(anon=True)  # anon=True for public data

In [22]:
# https://element84.com/earth-search/
# https://stacindex.org/catalogs/earth-search#/

client = Client.open(url="https://earth-search.aws.element84.com/v1")

In [13]:
# retry = Retry(total=5, backoff_factor=1, status_forcelist=[502, 503, 504], allowed_methods=None)
# stac_api_io = StacApiIO(max_retries=retry)
# client = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1", stac_io=stac_api_io)

In [14]:
client.conforms_to("ITEM_SEARCH")

True

In [ ]:
# client.add_conforms_to("ITEM_SEARCH")

In [4]:
# Define your polygon geometry as a GeoJSON dict:
# This is just an example polygon, replace with your own coordinates.
geometry = {
  "type": "Polygon",
  "coordinates": [
    [
      [-122.5, 37.7],
      [-122.3, 37.7],
      [-122.3, 37.8],
      [-122.5, 37.8],
      [-122.5, 37.7]
    ]
  ]
}

In [5]:
# search = client.search(
#     collections=["sentinel-2-l2a"],  # or "landsat-c2l2-sr" for Landsat Collection 2 Level-2, etc.
#     intersects=geometry,
#     datetime="2024-11-01/2024-12-10",  # optional temporal filter
#     limit=20
# )

# items = list(search.get_items())

# for item in items:
#     print(item.id, item.geometry)

In [32]:
search = client.search(
    collections=["sentinel-2-l2a"],
    # bbox=[-72.5,40.5,-72,41],
    intersects=geometry,
    # datetime="2024-11-01/2024-12-10",
    datetime=['2024-11-01T00:00:00Z', '2024-12-10T00:00:00Z'],
    max_items=20,
    sortby="properties.datetime",
)

print(f"{search.matched()} items found")

8 items found


In [36]:
item_collection = search.item_collection()
# item_collection.save_object('my_itemcollection.json')

In [37]:
item_collection

In [33]:
for item in search.items():
    print(item.id)
    # print(item.properties)
    print(item.properties['created'])
    print(item.properties['datetime'])
    print(item.properties['eo:cloud_cover'])
    print("="*100)

S2B_10SEG_20241104_0_L2A
2024-11-05T00:22:25.243Z
2024-11-04T19:04:05.835000Z
0.00777
S2A_10SEG_20241209_0_L2A
2024-12-09T23:45:33.953Z
2024-12-09T19:04:10.752000Z
0.71013
S2B_10SEG_20241204_0_L2A
2024-12-04T21:55:12.148Z
2024-12-04T19:04:09.853000Z
4.858421
S2A_10SEG_20241109_0_L2A
2024-11-10T00:18:47.417Z
2024-11-09T19:04:07.277000Z
19.783445
S2B_10SEG_20241124_0_L2A
2024-11-24T22:31:00.345Z
2024-11-24T19:04:10.837000Z
33.705357
S2B_10SEG_20241114_0_L2A
2024-11-14T23:21:49.936Z
2024-11-14T19:04:05.663000Z
48.525691
S2A_10SEG_20241119_0_L2A
2024-11-20T00:06:12.904Z
2024-11-19T19:04:09.340000Z
53.079361
S2A_10SEG_20241129_0_L2A
2024-11-30T00:04:56.797Z
2024-11-29T19:04:08.589000Z
57.964945


In [38]:
search = client.search(
    collections=["sentinel-2-l2a"],
    intersects=geometry,
    datetime=['2024-11-01T00:00:00Z', '2024-12-10T00:00:00Z'],
    max_items=20,
    sortby=[
        {"direction": "desc", "field": "properties.datetime"},
        {"direction": "asc", "field": "id"},
        {"direction": "asc", "field": "collection"},
    ]
)

## collection search

In [8]:
collection_search = client.collection_search(q='"sentinel-2" OR "sentinel-1"')

print(f"{collection_search.matched()} collections found")

/home/apolo/anaconda3/envs/geospatial/lib/python3.11/site-packages/pystac_client/collection_search.py:292: UserWarning: Server does not conform to COLLECTION_SEARCH. Filtering will be performed client-side where only bbox, datetime, and q arguments are supported
  warnings.warn(


5 collections found


In [9]:
# collection_search.collection_list_as_dict()
# collection_search.collection_list()

for collection in collection_search.collections():
    print(collection.id)

sentinel-2-pre-c1-l2a
sentinel-2-l2a
sentinel-2-l1c
sentinel-2-c1-l2a
sentinel-1-grd


In [15]:
catalog = Client.open('https://planetarycomputer.microsoft.com/api/stac/v1')

results = catalog.collection_search(
    q="biomass",
    datetime="2022/.."
)

/home/apolo/anaconda3/envs/geospatial/lib/python3.11/site-packages/pystac_client/collection_search.py:292: UserWarning: Server does not conform to COLLECTION_SEARCH. Filtering will be performed client-side where only bbox, datetime, and q arguments are supported
  warnings.warn(


In [18]:
for collection in results.collections():
    print(collection.id)

fia
modis-13Q1-061
modis-13A1-061
sentinel-3-olci-lfr-l2-netcdf


## Loading data

In [ ]:
with rasterio.open(item.assets["data"].href) as dataset:
    rasterio.plot.show(dataset)

In [ ]:
catalog = Client.open(...)

query = catalog.search(...)

In [ ]:
xx = odc.stac.load(
    query.get_items(),
    bands=["red", "green", "blue"],
    resolution=100,
)
xx.red.plot.imshow(col="time")